In [29]:
import os
import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

In [30]:
import pandas as pd 

train_org = pd.read_csv('data/train.csv') 
test_org = pd.read_csv('data/test.csv')

In [31]:
train_df = train_org.copy()
test_df = test_org.copy()

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})' 

train_df[['연', '월', '일', '시간']] = train_org['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다 
train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다 

# 해당 과정을 test_x에 대해서도 반복해줍니다 
test_df[['연', '월', '일', '시간']] = test_org['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류',\n       '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간'], dtype='object')"

In [32]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_org['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test_org['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반',\n       '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간', '도시', '구', '동'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시',\n       '구', '동'],\n      dtype='object')"

In [33]:
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_org['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

test_df[['도로형태1', '도로형태2']] = test_org['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반', '가해운전자 차종',\n       '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별',\n       '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수', 'ECLO', '연',\n       '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '동',\n       '도로형태1', '도로형태2'],\n      dtype='object')"

In [34]:
test_x = test_df.drop(columns=['ID']).copy()
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()

In [35]:
# 전처리한 train / test 저장
train_df.to_csv('data/modified_train.csv')
test_df.to_csv('data/modified_test.csv')

In [39]:
display(len(train_df.columns))
print(train_x['기상상태'].value_counts())
print(train_x['요일'].value_counts())

29

기상상태
맑음    36181
비      2627
흐림      729
기타       56
안개        8
눈         8
Name: count, dtype: int64
요일
금요일    6179
화요일    6023
수요일    5969
월요일    5895
목요일    5759
토요일    5670
일요일    4114
Name: count, dtype: int64


# 전처리 2


In [10]:
train_df =pd.read_csv('data/modified_train.csv')
test_df = pd.read_csv('data/modified_test.csv')

In [11]:
light_df = pd.read_csv('data/external_open/대구 보안등 정보.csv', encoding='cp949')[['설치개수', '소재지지번주소']]

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

light_df[['도시', '구', '동', '번지']] = light_df['소재지지번주소'].str.extract(location_pattern)
light_df = light_df.drop(columns=['소재지지번주소', '번지'])

light_df = light_df.groupby(['도시', '구', '동']).sum().reset_index()
light_df.reset_index(inplace=True, drop=True)

C:\Users\HongWonJin\AppData\Local\Temp\ipykernel_46356\362901134.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  light_df = pd.read_csv('data/external_open/대구 보안등 정보.csv', encoding='cp949')[['설치개수', '소재지지번주소']]


In [12]:
parking_df = pd.read_csv('data/external_open/대구 주차장 정보.csv', encoding='cp949')[['소재지지번주소', '급지구분']]
parking_df = pd.get_dummies(parking_df, columns=['급지구분'])

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

parking_df[['도시', '구', '동', '번지']] = parking_df['소재지지번주소'].str.extract(location_pattern)
parking_df = parking_df.drop(columns=['소재지지번주소', '번지'])

parking_df = parking_df.groupby(['도시', '구', '동']).sum().reset_index()
parking_df.reset_index(inplace=True, drop=True)

In [13]:
child_area_df = pd.read_csv('data/external_open/대구 어린이 보호 구역 정보.csv', encoding='cp949').drop_duplicates()[['소재지지번주소']]
child_area_df['cnt'] = 1

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

child_area_df[['도시', '구', '동', '번지']] = child_area_df['소재지지번주소'].str.extract(location_pattern)
child_area_df = child_area_df.drop(columns=['소재지지번주소', '번지'])

child_area_df = child_area_df.groupby(['도시', '구', '동']).sum().reset_index()
child_area_df.reset_index(inplace=True, drop=True)

In [14]:
# train_df와 test_df에, light_df와 child_area_df, parking_df를 merge하세요.
train_df = pd.merge(train_df, light_df, how='left', on=['도시', '구', '동'])
train_df = pd.merge(train_df, child_area_df, how='left', on=['도시', '구', '동'])
train_df = pd.merge(train_df, parking_df, how='left', on=['도시', '구', '동'])

test_df = pd.merge(test_df, light_df, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, child_area_df, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, parking_df, how='left', on=['도시', '구', '동'])

In [15]:
# 없는 값 평균으로 채우기

train_df['cnt'] = train_df['cnt'].fillna(9)
train_df['설치개수'] = train_df['설치개수'].fillna(train_df['설치개수'].mean())

train_df['급지구분_1'] = train_df['급지구분_1'].fillna(train_df['급지구분_1'].mean())
train_df['급지구분_2'] = train_df['급지구분_2'].fillna(train_df['급지구분_2'].mean())
train_df['급지구분_3'] = train_df['급지구분_3'].fillna(train_df['급지구분_3'].mean())

test_df['cnt'] = test_df['cnt'].fillna(0)
test_df['설치개수'] = test_df['설치개수'].fillna(test_df['설치개수'].mean())

test_df['급지구분_1'] = test_df['급지구분_1'].fillna(test_df['급지구분_1'].mean())
test_df['급지구분_2'] = test_df['급지구분_2'].fillna(test_df['급지구분_2'].mean())
test_df['급지구분_3'] = test_df['급지구분_3'].fillna(test_df['급지구분_3'].mean())

In [16]:
train_df.to_csv('data/TE_add_columns_train.csv')
test_df.to_csv('data/TE_add_columns_test.csv')

In [17]:
len(train_df.columns)

35

In [18]:
test_x = test_df.drop(columns=['ID']).copy()
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()

In [19]:
from category_encoders.target_encoder import TargetEncoder

# 타겟인코딩 진행
categorical_features = list(train_x.dtypes[train_x.dtypes == "object"].index)
# 추출된 문자열 변수 확인
display(categorical_features)

for i in categorical_features:
    te = TargetEncoder(cols=[i])
    train_x[i] = te.fit_transform(train_x[i], train_y)
    test_x[i] = te.transform(test_x[i])

['요일', '기상상태', '노면상태', '사고유형', '도시', '구', '동', '도로형태1', '도로형태2']

In [20]:
print(len(train_df.columns))

35


In [21]:
train_x = train_x.drop(columns='Unnamed: 0')
test_x = test_x.drop(columns='Unnamed: 0')

display(train_x)
display(test_x)

,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2,설치개수,cnt,급지구분_1,급지구분_2,급지구분_3
0,4.627926,4.712888,4.712878,3.817650,2019,1,1,0,4.726704,4.541610,4.282449,4.671841,4.599599,391.000000,2.0,11.000000,0.00000,0.000000
1,4.627926,4.779150,4.712878,3.817650,2019,1,1,0,4.726704,4.618441,4.738938,4.671841,4.599599,932.000000,9.0,0.000000,1.00000,3.000000
2,4.627926,4.712888,4.712878,3.817650,2019,1,1,1,4.726704,4.727300,4.842715,4.671841,4.599599,473.000000,5.0,2.549961,4.50121,3.164217
3,4.627926,4.712888,4.712878,4.944597,2019,1,1,2,4.726704,4.687669,4.208920,4.671841,4.599599,534.000000,11.0,0.000000,9.00000,5.000000
4,4.627926,4.712888,4.712878,4.944597,2019,1,1,4,4.726704,4.889534,4.549091,4.671841,4.599599,2057.000000,9.0,0.000000,1.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,4.600906,4.712888,4.712878,4.944597,2021,12,31,19,4.726704,4.727300,4.913420,4.882281,5.006142,881.939427,1.0,2.549961,4.50121,3.164217
39605,4.600906,4.712888,4.712878,4.944597,2021,12,31,19,4.726704,4.618441,4.293963,4.671841,4.599599,843.000000,9.0,0.000000,0.00000,5.000000
39606,4.600906,4.712888,4.712878,4.944597,2021,12,31,21,4.726704,4.618441,4.648867,4.882281,5.006142,164.000000,9.0,0.000000,1.00000,0.000000
39607,4.600906,4.712888,4.712878,4.944597,2021,12,31,22,4.726704,4.618441,4.672002,4.065962,4.599599,210.000000,9.0,0.000000,0.00000,1.000000


,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2,설치개수,cnt,급지구분_1,급지구분_2,급지구분_3
0,4.920811,4.712888,4.712878,3.817650,2022,1,1,1,4.726704,4.727300,4.881657,4.882281,5.006142,700.000000,5.0,2.503929,4.49275,3.18539
1,4.920811,4.712888,4.712878,3.817650,2022,1,1,1,4.726704,4.727300,4.563008,4.671841,4.599599,852.041626,10.0,0.000000,0.00000,2.00000
2,4.920811,4.712888,4.712878,4.944597,2022,1,1,4,4.726704,4.727300,4.945578,4.882281,5.006142,852.041626,1.0,2.503929,4.49275,3.18539
3,4.920811,4.712888,4.712878,4.944597,2022,1,1,4,4.726704,4.727300,4.438172,4.671841,4.599599,852.041626,7.0,0.000000,2.00000,1.00000
4,4.920811,4.712888,4.712878,4.944597,2022,1,1,6,4.726704,4.618441,4.738938,4.882281,5.006142,932.000000,0.0,0.000000,1.00000,3.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,4.920811,4.712888,4.712878,4.944597,2022,12,31,18,4.726704,4.623681,4.492281,4.671841,6.555670,5377.000000,26.0,20.000000,1.00000,0.00000
10959,4.920811,4.712888,4.712878,4.944597,2022,12,31,18,4.726704,4.727300,4.928144,4.671841,4.599599,852.041626,5.0,2.503929,4.49275,3.18539
10960,4.920811,4.712888,4.712878,4.944597,2022,12,31,20,4.726704,4.727300,5.355036,4.671841,4.599599,852.041626,0.0,2.503929,4.49275,3.18539
10961,4.920811,4.712888,4.712878,4.944597,2022,12,31,20,4.726704,4.727300,5.079498,4.882281,4.863035,852.041626,7.0,2.503929,4.49275,3.18539


In [27]:
print(train_x['기상상태'].value_counts())

기상상태
4.712888    36181
4.895699     2627
4.779150      729
4.940585       56
4.905702        8
4.703162        8
Name: count, dtype: int64


In [ ]:
//////////////////////////////////////////////////////////////////
# 시 군 구 를 위도와 경도로 합쳐서 country_wide데이터까지 조인

In [20]:
import pandas as pd


countrywide = pd.read_csv('data/external_open/countrywide_accident.csv')
countrywide.shape
countrywide.head(1)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,COUNTRYWIDE_ACCIDENT_000000,2019-01-01 00,화요일,맑음,서울특별시 강서구 방화동,교차로 - 교차로횡단보도내,건조,차대사람,횡단중,보행자보호의무위반,...,상해없음,보행자,남,40세,경상,0,0,1,0,3


In [21]:
from geopy.geocoders import Nominatim

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd

crd = geocoding("포천시 소흘읍")
print(crd['lat'])
print(crd['lng'])


37.8039536
127.149641


In [57]:
countrywide['시군구']

location_pattern = r'(\S+) (\S+) (\S+)'

a = countrywide['시군구'].str.extract(location_pattern)[[1,2]]

# result_list = list(a[1] + ' ' + a[2])

In [58]:
geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)

def get_lat_lon(address):
    location = geolocoder.geocode(address)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

In [65]:
countrywide[['위도', '경도']] = a.apply(lambda row: pd.Series(get_lat_lon(f"{row[1]} {row[2]}")), axis=1)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%EC%95%88%EC%82%B0%EC%8B%9C+%EC%83%81%EB%A1%9D%EA%B5%AC&format=json&limit=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001869EC41BA0>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
display(countrywide['위도'])
display(countrywide['경도'])